# AWQ on Vicuna

In this notebook, we use Vicuna model to demonstrate the performance of AWQ on instruction-tuned models. We implement AWQ real-INT4 inference kernels, which are wrapped as Pytorch modules and can be easily used by existing models. We also provide a simple example to show how to use AWQ to quantize a model and save/load the quantized model checkpoint.

In order to run this notebook, you need to install the following packages:
- [AWQ](https://github.com/mit-han-lab/llm-awq)
- [Pytorch](https://pytorch.org/)
- [Accelerate](https://github.com/huggingface/accelerate)
- [Transformers](https://github.com/huggingface/transformers)

In [1]:
from awq.models.auto import AutoAWQForCausalLM
from transformers import AutoTokenizer
from tinychat.demo import gen_params, stream_output
from tinychat.stream_generators import StreamGenerator
from tinychat.modules import make_quant_norm, make_quant_attn, make_fused_mlp
from tinychat.utils.prompt_templates import get_prompter
import os
# This demo only support single GPU for now
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
model_path = 'vicuna-7b-v1.5-awq'
quant_file = 'awq_model_w4_g128.pt'

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
model = AutoAWQForCausalLM.from_quantized(model_path, quant_file)

Replacing layers...: 100%|██████████| 32/32 [00:02<00:00, 11.85it/s]


In [3]:
make_quant_attn(model.model, "cuda:0")
make_quant_norm(model.model)
make_fused_mlp(model.model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): QuantLlamaAttention(
          (qkv_proj): WQLinear(in_features=4096, out_features=12288, bias=False, w_bit=4, group_size=128)
          (o_proj): WQLinear(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
          (rotary_emb): QuantLlamaRotaryEmbedding()
        )
        (mlp): QuantLlamaMLP(
          (down_proj): WQLinear(in_features=11008, out_features=4096, bias=False, w_bit=4, group_size=128)
        )
        (input_layernorm): FTLlamaRMSNorm()
        (post_attention_layernorm): FTLlamaRMSNorm()
      )
    )
    (norm): FTLlamaRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)

In [4]:
model_prompter = get_prompter(model, model_path)
stream_generator = StreamGenerator
count = 0
while True:
    # Get input from the user
    input_prompt = input("USER: ")
    if input_prompt == "":
        print("EXIT...")
        break
    model_prompter.insert_prompt(input_prompt)
    output_stream = stream_generator(model, tokenizer, model_prompter.model_input, gen_params, device="cuda:0")
    outputs = stream_output(output_stream)    
    model_prompter.update_template(outputs)
    count += 1

ASSISTANT: Sure! Here are some popular tourist attractions in Boston:

1. Freedom Trail - a 2.5-mile walking trail that takes you through some of the most important historical sites in Boston, including Paul Revere's House, the Old North Church, and the site of the Boston Massacre.
2. Fenway Park - home to the Boston Red Sox baseball team, this historic ballpark is one of the oldest in Major League Baseball.
3. Museum of Fine Arts - one of the largest art museums in the country, with a collection of over 450,000 works of art from around the world.
4. Boston Harbor Islands National Recreation Area - a group of islands located just offshore from downtown Boston that offer stunning views of the city skyline and easy access to outdoor recreational activities like hiking and kayaking.
5. New England Aquarium - one of the oldest and largest aquariums in the United States, featuring a wide variety of marine life, including giant whales and colorful fish.
6. The USS Constitution Museum - locat